In [1]:
import pandas as pd

competitor_rankings_data = pd.read_csv('C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/csv_files/competitor_rankings_details.csv')
competitor_rankings_data

,competitor_rank,competitor_rank_movement,competitor_rank_points,competitor_rank_competitions_played,competitor_id
0,1,0,9950,22,sr:competitor:49363
1,1,0,9950,22,sr:competitor:51836
2,3,0,7590,25,sr:competitor:14898
3,4,0,7590,26,sr:competitor:637970
4,5,1,6180,20,sr:competitor:36593
...,...,...,...,...,...
995,496,-5,129,12,sr:competitor:831284
996,497,42,129,16,sr:competitor:203959
997,498,-8,128,9,sr:competitor:784280
998,499,-6,128,14,sr:competitor:840734


save data in postgresql database

In [2]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


Creating DataBase:

In [3]:
pip install toml

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import psycopg2
import streamlit as st
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

import json
import toml

import sys
import os

# Load environment
try:
    with open("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/config.json", "r") as f:
        config = json.load(f)

    env = config["environment"] # 'local' or 'cloud'
except FileNotFoundError:
    raise FileNotFoundError("⚠️ 'config.json' not found. Please check the path.")
except json.JSONDecodeError:
    raise ValueError("⚠️ 'config.json' is not a valid JSON file.")
except KeyError:
    raise KeyError("⚠️ 'environment' key not found in config.json.")

# Load secrets from your custom toml file
local_secrets = toml.load("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar/.pwd/pwd.toml")



# Append the directory where db_utils.py is located
sys.path.append(os.path.abspath('C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project1/Tennis_Data_SportRadar'))


from db_utils import connect_to_postgres, connect_to_sportradar

# Connect to default DB and create target DB if needed
my_db_connection = connect_to_postgres(env, local_secrets) #connect to existing database first

# Set autocommit so CREATE DATABASE works
my_db_connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Create cursor and check for database existence
mycursor = my_db_connection.cursor()
mycursor.execute("SELECT * FROM pg_database WHERE datname='sportradar'")
exists = mycursor.fetchone()

if not exists:
    mycursor.execute("CREATE DATABASE sportradar")   #create target DB
    print("Database 'sportradar' created")
else:
    print("Database 'sportradar' already exists")

mycursor.close()
my_db_connection.close()




# Reconnect to the newly ensured 'sportradar' database
my_db_connection = connect_to_sportradar(env, local_secrets)

my_db_connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

#I need to connect the db and the python:
mycursor = my_db_connection.cursor()

print("Connected to 'sportradar' database.")

Database 'sportradar' already exists
Connected to 'sportradar' database.


Creating competition Table

In [5]:
competitor_rankings_table_sql = """create table if not exists competitor_rankings_table(
competitor_rank_id SERIAL PRIMARY KEY, 
competitor_rank int NOT NULL, 
competitor_rank_movement int NOT NULL,
competitor_rank_points int NOT NULL, 
competitor_rank_competitions_played int NOT NULL, 
competitor_id varchar(50) NOT NULL,
FOREIGN KEY(competitor_id) references competitors_table(competitor_id))"""

#execute:
mycursor.execute(competitor_rankings_table_sql)

print("Competitor Rankings Table has been created")

Competitor Rankings Table has been created


Fetching Column Name from competitor_rankings_table:

In [6]:
column_name = """SELECT column_name FROM 
        information_schema.columns WHERE table_name = 'competitor_rankings_table';"""
mycursor.execute(column_name)
fetching_column_names = mycursor.fetchall()
fetching_column_names

[('competitor_rank_id',),
 ('competitor_rank',),
 ('competitor_rank_movement',),
 ('competitor_rank_points',),
 ('competitor_rank_competitions_played',),
 ('competitor_id',)]

insert values into competitor rankings table:

In [7]:
competitor_rankings_insert_table_sql = """Insert into competitor_rankings_table(
competitor_rank, 
competitor_rank_movement,
competitor_rank_points,
competitor_rank_competitions_played,
competitor_id) 
values (%s, %s, %s, %s, %s);"""

In [8]:
mycursor.execute("SELECT competitor_id FROM competitors_table;")
existing_ids = set(row[0] for row in mycursor.fetchall())

# Insert only valid rows
for index, row in competitor_rankings_data.iterrows():
    if row['competitor_id'] in existing_ids:
        mycursor.execute(competitor_rankings_insert_table_sql, tuple(row))
        print(row) #We will get the o/p in the format of dictionary with column names as keys.
        
        print("Inserting:", tuple(row))  #The iter row (only values) will converts into python tuple 
        print("----------------------\n")
    else:
        print(f"Skipping insert for missing competitor_id: {row['competitor_id']}")

print("Data inserted  into the competitor_rankings_table using iterrows()")

competitor_rank                                          1
competitor_rank_movement                                 0
competitor_rank_points                                9950
competitor_rank_competitions_played                     22
competitor_id                          sr:competitor:49363
Name: 0, dtype: object
Inserting: (1, 0, 9950, 22, 'sr:competitor:49363')
----------------------

competitor_rank                                          1
competitor_rank_movement                                 0
competitor_rank_points                                9950
competitor_rank_competitions_played                     22
competitor_id                          sr:competitor:51836
Name: 1, dtype: object
Inserting: (1, 0, 9950, 22, 'sr:competitor:51836')
----------------------

competitor_rank                                          3
competitor_rank_movement                                 0
competitor_rank_points                                7590
competitor_rank_competitions_played  

Fetching Data from competitor_rankings_table:

In [9]:
#execute:
mycursor.execute("Select * from competitor_rankings_table")

fetching_Data = mycursor.fetchall()
fetching_Data

[(1, 1, 0, 9950, 22, 'sr:competitor:49363'),
 (2, 1, 0, 9950, 22, 'sr:competitor:51836'),
 (3, 3, 0, 7590, 25, 'sr:competitor:14898'),
 (4, 4, 0, 7590, 26, 'sr:competitor:637970'),
 (5, 5, 1, 6180, 20, 'sr:competitor:36593'),
 (6, 6, 1, 6090, 19, 'sr:competitor:52293'),
 (7, 7, 3, 5675, 16, 'sr:competitor:15568'),
 (8, 8, 3, 5675, 19, 'sr:competitor:16160'),
 (9, 9, -1, 5580, 22, 'sr:competitor:95801'),
 (10, 10, -1, 5520, 20, 'sr:competitor:15310'),
 (11, 11, -6, 5460, 19, 'sr:competitor:87690'),
 (12, 12, 0, 4710, 13, 'sr:competitor:124658'),
 (13, 13, 0, 4700, 25, 'sr:competitor:16608'),
 (14, 14, 0, 4460, 33, 'sr:competitor:59131'),
 (15, 15, 0, 4325, 31, 'sr:competitor:108099'),
 (16, 16, 0, 4000, 27, 'sr:competitor:16151'),
 (17, 17, 0, 3820, 26, 'sr:competitor:53785'),
 (18, 18, 5, 3031, 35, 'sr:competitor:23899'),
 (19, 19, 2, 2955, 24, 'sr:competitor:38181'),
 (20, 20, 5, 2855, 36, 'sr:competitor:52065'),
 (21, 21, 3, 2820, 20, 'sr:competitor:15548'),
 (22, 22, -4, 2780, 26, '

close cursor & db connection:

In [10]:
mycursor.close()
my_db_connection.close()